## Assignment 1
- Darpan Gaur        CO21BTECH11004
- Aditya Bacharwar      ES21BTECH11003
- Bapatu Manoj Kumar Reddy      ES21BTECH11010

In [55]:
# import libraries
import numpy as np
import pandas as pd

Input data format: Input csv file with m+2 rows and n+1 columns
- The first row excluding the last element is the inital feasible point z of lenght n
- The second row excluding the last element is the cost vector c of length n
- The last column excluding the top two elements is the constraint vector b of lenght m
- Rows third to m+2 and columns one to n is the matrix A of size m*n

In [56]:
def get_input(input_file_path):
    '''
    Read input file and return z, c, b, A
    '''
    # read input file
    df = pd.read_csv(input_file_path, header=None)

    # z -> first row, except last column
    z = df.iloc[0, :-1].values

    # c -> second row, except last column
    c = df.iloc[1, :-1].values

    # b -> next m rows, last column
    b = df.iloc[2:, -1].values

    # A -> next m rows, except last column
    A = df.iloc[2:, :-1].values

    return z, c, b, A

In [57]:
def compute_constrainsts(z, A, b, tight=True):
    '''
    Compute tight / untight constraints

    Returns:
        A_tight, b_tight or A_untight, b_untight
    '''
    rows, constraints = [], []
    for i in range(A.shape[0]):
        row = A[i]
        constraint = b[i]
        if (np.dot(row, z) == constraint) == tight:
            rows.append(row)
            constraints.append(constraint)
    return np.array(rows), np.array(constraints)

In [58]:
def compute_directions(A):
    '''
    Compute directions from tight constraints
    '''
    
    return -np.linalg.inv(A)


In [59]:
def find_possible_directions(c, directions):
    '''
    Find the first positive direction
    '''
    
    cost = np.dot(c, directions)
    positive_cost_indices = np.where(cost > 0)[0]
    if len(positive_cost_indices) == 0:
        return None, None
    return positive_cost_indices[0], directions[:, positive_cost_indices[0]]
    

In [60]:
def check_unboundedness(A, direction):
    '''
    Check if the problem is unbounded in the direction
    '''
    
    try:
        if (np.where(A@direction > 0)[0].size == 0):
            raise Exception('Problem is unbounded')
    except:
        return True
        

In [61]:
def simplex_algorithm(z, c, b, A, max_iters = 1000):
    '''
    Simplex algorithm for maximizing objective function
    '''

    z_temp = z
    iter = 0
    try:
        while iter<max_iters:
            A_tight, b_tight = compute_constrainsts(z_temp, A, b)
            A_untight, b_untight = compute_constrainsts(z_temp, A, b, tight=False)
            # print(A_tight, b_tight)
            directions = compute_directions(A_tight)
            direction_index, direction = find_possible_directions(c, directions)
            if direction is None:
                break
            if (check_unboundedness(A, direction)):
                raise Exception('Problem is unbounded')
            eta = (b_untight - np.dot(A_untight, z_temp)) / np.dot(A_untight, direction)
            eta_min = np.min(eta[eta>0])
            z_temp = z_temp + eta_min * direction
            iter += 1
    except Exception as e:
        print(e)
        return None, None
    return z_temp, np.dot(c, z_temp)

    

In [62]:
# set input file path
input_file_path = './input5.csv'

# get input
z, c, b, A = get_input(input_file_path)

# print input
print('z:', z)
print('c:', c)
print('b:', b)
print('A:', A)

z: [10  0]
c: [4 5]
b: [10 42  0  0]
A: [[ 1  1]
 [ 3  7]
 [-1  0]
 [ 0 -1]]


In [63]:
# run simplex algorithm
optimal_z, optimal_cost = simplex_algorithm(z, c, b, A)

# print output
print('optimal_z:', optimal_z)
print('optimal_cost:', optimal_cost)

optimal_z: [7. 3.]
optimal_cost: 43.0
